In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 50)
pd.set_option('display.max_colwidth', 150)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import seaborn as sns
sns.set_theme()

In [2]:
try:
    from cellacdc import cca_functions
    from cellacdc import myutils
except FileNotFoundError:
    # Check if user has developer version --> add the Cell_ACDC/cellacdc
    # folder to path and import from thre
    sys.path.insert(0, '../cellacdc/')
    from cellacdc import cca_functions
    from cellacdc import myutils

In [ ]:
def load_acdc_output_only(
    file_names,
    image_folders,
    positions
):
    """
    Function to load only the acdc output.
    Use when fluorescent file is too big to load into RAM.
    #TODO: move to cca_functions
    """
    overall_df = pd.DataFrame()
    for file_idx, file in enumerate(file_names):
        for pos_idx, pos_dir in enumerate(image_folders[file_idx]):
            try:
                cc_stage_path = glob.glob(os.path.join(f'{pos_dir}', '*acdc_output.csv'))[0]
            except IndexError:
                cc_stage_path = glob.glob(os.path.join(f'{pos_dir}', '*cc_stage.csv'))[0]
            temp_df = pd.read_csv(cc_stage_path)
            temp_df['max_frame_pos'] = temp_df.frame_i.max()
            temp_df['file'] = file
            temp_df['selection_subset'] = file_idx
            temp_df['position'] = positions[file_idx][pos_idx]
            temp_df['directory'] = pos_dir
            overall_df = pd.concat([overall_df, temp_df])
    return overall_df

In [ ]:
%load_ext autoreload
%autoreload 2

# configurations
- follow the file selection dialog:
    - select microscopy folder in first step
    - select positions of the selected folder in second step
- repeat to add more positions to the analysis
- positions selected within one iteration of the dialog will be pooled together in the following analyses

In [ ]:
data_dirs, positions = cca_functions.configuration_dialog()
file_names = [os.path.split(path)[-1] for path in data_dirs]
image_folders = [[os.path.join(data_dir, pos_str, 'Images') for pos_str in pos_list] for pos_list, data_dir in zip(positions, data_dirs)]
# determine available channels based on first(!) position.
# Warn user if one or more of the channels are not available for some positions
first_pos_dir = os.path.join(data_dirs[0], positions[0][0], 'Images')
first_pos_files = myutils.listdir(first_pos_dir)
channels, warn = cca_functions.find_available_channels(first_pos_files, first_pos_dir)

# load data and perform all needed calculations on image data

In [ ]:
data_dirs, positions, file_names, image_folders

In [ ]:
"""
overall_df, is_timelapse_data, is_zstack_data = cca_functions.calculate_downstream_data(
    file_names,
    image_folders,
    positions,
    channels, 
    force_recalculation=False
)
"""
overall_df = load_acdc_output_only(
    file_names,
    image_folders,
    positions
)
is_timelapse_data = True

In [ ]:
overall_df.shape

In [ ]:
# if cell cycle annotations were performed in ACDC, extend the dataframe by a join on each cells relative cell
if 'cell_cycle_stage' in overall_df.columns:
    overall_df_with_rel = cca_functions.calculate_relatives_data(overall_df, channels)
# If working with timelapse data build dataframe grouped by phases
group_cols = [
    'Cell_ID', 'generation_num', 'cell_cycle_stage', 'relationship', 'position', 'file', 
    'max_frame_pos', 'selection_subset', 'max_t'
]
# calculate data grouped by phase only in the case, that timelapse data is available
if is_timelapse_data and 'max_t' in overall_df_with_rel.columns:
    phase_grouped = cca_functions.calculate_per_phase_quantities(overall_df_with_rel, group_cols, channels)
    # append phase-grouped data to overall_df_with_rel
    overall_df_with_rel = overall_df_with_rel.merge(
        phase_grouped,
        how='left',
        on=group_cols
    )
    overall_df_with_rel['time_in_phase'] = overall_df_with_rel['frame_i'] - overall_df_with_rel['phase_begin'] + 1
    overall_df_with_rel['time_in_cell_cycle'] = overall_df_with_rel.groupby(['Cell_ID', 'generation_num', 'position', 'file'])['frame_i'].transform('cumcount') + 1

# Plot gallery - timelapse data

## Symmetric cell division: cell size at birth vs. cell cycle length and growth in first frames

In [ ]:
def try_find_entry(x, idx):
    """
    helper function to include a try except statement within groupby aggregation
    """
    try:
        return x.iloc[idx]
    except IndexError:
        return 0

In [ ]:

symm_plot_data = overall_df.copy()
grouping_cols = [
    'Cell_ID', 
    'Cell_ID_tree', 
    'parent_ID_tree', 
    'file', 
    'position'
]
symm_plot_data['rolling_avg_area'] = (
    symm_plot_data.groupby(grouping_cols)['cell_area_pxl']
    .transform(lambda x: x.rolling(window=3, center=True).mean())
)
symm_plot_data = symm_plot_data.sort_values('frame_i')
cc_frames = symm_plot_data.groupby(grouping_cols).agg(
    birth_frame = ('frame_i', min),
    cc_end_frame = ('frame_i', max),
    birth_area = ('cell_area_pxl', lambda x: x.iloc[0]),
    mean_area_first_5 = ('cell_area_pxl', lambda x: x.iloc[:5].mean()),
    mean_area_first_10 = ('cell_area_pxl', lambda x: x.iloc[:10].mean()),
    avg_area_first3 = ('rolling_avg_area', lambda x: try_find_entry(x, 1)),
    avg_area_9to11 = ('rolling_avg_area', lambda x: try_find_entry(x, 9))
).reset_index()


In [ ]:
cc_frames['cc_length'] = cc_frames['cc_end_frame'] - cc_frames['birth_frame']
# filter for birth_frame>0 (birth of cells present at beginning of experiment cannot be observed)
cc_frames = cc_frames.loc[cc_frames['birth_frame']>0]
# calculate last frames per experiment and join this information with data
last_frames_per_experiment = overall_df.groupby(['file', 'position']).agg(
    last_frame = ('frame_i', max)
).reset_index()
cc_frames = pd.merge(cc_frames, last_frames_per_experiment, how='left', on=['position', 'file'])
# filter out rows where cell cycle "ends" on last frame (could just be bc of end of experiment)
cc_frames = cc_frames[cc_frames.last_frame!=cc_frames.cc_end_frame]
# filter out rows with cell cycle length 0 (those are rows representing non-observable S phases)
cc_frames = cc_frames[cc_frames.cc_length>0]
# calculate growth at beginning of cell cycle by subtracting sliding avg of first 3 frames from sliding avg within frames 9 to 11
cc_frames['growth_in_first_10_frames'] = cc_frames.avg_area_9to11 - cc_frames.avg_area_first3

In [ ]:
cc_frames.head()

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Correlation of cell size at birth and cell cycle length')
sns.regplot(
    data=cc_frames,
    x = 'birth_area',
    y = 'cc_length'
)
plt.ylabel('Length of cell cycle [frames]')
plt.xlabel('Area of cell at birth [pixels]')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Correlation of cell size at birth and growth in the first 10 frames')
sns.regplot(
    data=cc_frames,
    x = 'birth_area',
    y = 'growth_in_first_10_frames'
)
plt.ylabel('Change of area during 10 first frames [pixels]')
plt.xlabel('Area of cell at birth [pixels]')
plt.show()

## Distribution of cell cycle lengths

In [ ]:
complete_cc_data = overall_df_with_rel[overall_df_with_rel.complete_cycle==1]
cc_lengths = complete_cc_data.groupby(['Cell_ID', 'generation_num', 'file', 'position'])['time_in_cell_cycle'].max()
sns.histplot(cc_lengths)
plt.show()

## Fluorescence signal per volume in G1 aligned at cell birth
- fluorescence concentration per cell over time
- 1st-generation G1
- align at birth

In [ ]:
# set this to match with channel of interest
ch_name = 'mCitrine'
# filter for relevant rows (first gen G1 cells)
plot_data7 = overall_df_with_rel[
    (overall_df_with_rel.cell_cycle_stage=='G1') &
    (overall_df_with_rel.generation_num==1) &
    (overall_df_with_rel.is_history_known) &
    (overall_df_with_rel.complete_phase)
]
# select columns of interest for the plot
plot_data7 = plot_data7[
    ['file', 'position', 'frame_i', 'Cell_ID', 'phase_begin', 'generation_num', f'{ch_name}_corrected_concentration']
]
# calculate "time in phase" column for x-axis
plot_data7['time_in_phase'] = plot_data7['frame_i'] - plot_data7['phase_begin']
# calculate a unique cell id accross files by just appending file, pos & cell id
plot_data7['f_pos_cell_id'] = plot_data7.apply(lambda x: f'{x["file"]}_{x["position"]}_Cell_{x["Cell_ID"]}_Gen_{int(x["generation_num"])}', axis=1)

In [ ]:
# Generate figures (aggregated, single traces, combined
sns.set_theme(context='talk', font_scale=1.15)
sns.set_style("whitegrid", {"grid.color": ".95"})
fig, axs = plt.subplots(ncols=3, figsize=(30,10), sharey=False)
sns.lineplot(
    data=plot_data7,
    x="time_in_phase", 
    y=f'{ch_name}_corrected_concentration',
    ci=95,
    ax=axs[0]
)
sns.lineplot(
    data=plot_data7,
    x="time_in_phase", 
    y=f'{ch_name}_corrected_concentration',
    estimator=None,
    units='f_pos_cell_id',
    ax=axs[1],
    lw=0.5,
    #alpha=0.5
)
sns.lineplot(
    data=plot_data7,
    x="time_in_phase", 
    y=f'{ch_name}_corrected_concentration',
    ci=95,
    ax=axs[2]
)
sns.lineplot(
    data=plot_data7,
    x="time_in_phase", 
    y=f'{ch_name}_corrected_concentration',
    estimator=None,
    units='f_pos_cell_id',
    ax=axs[2],
    lw=0.5,
    #alpha=0.5
)
axs[0].set_ylabel(f"Amount per Volume of {ch_name} [a.u.]")
axs[1].set_ylabel(f"Amount per Volume of {ch_name} [a.u.]")
axs[2].set_ylabel(f"Amount per Volume of {ch_name} [a.u.]")
#plt.savefig('../figures/firstgen_g1_concentration.png', dpi=300)
plt.show()

## (Volume) growth in G1 vs. mother+daughter growth in S (1st generation)

In [ ]:
# obtain table where one cell cycle is represented by one row: 
# first set of columns (like phase_length, growth...) for G1, second set of cols for S
complete_cc_data = phase_grouped[phase_grouped.all_complete==1]
s_data = complete_cc_data[complete_cc_data.cell_cycle_stage=="S"]
g1_data = complete_cc_data[complete_cc_data.cell_cycle_stage=="G1"]
plot_data2 = g1_data.merge(
    s_data, on=['Cell_ID', 'generation_num', 'position'], how='inner', suffixes=('_g1','_s')
)
plot_data2 = plot_data2[plot_data2.generation_num==1]
plot_data2['combined_motherbud_growth'] = plot_data2['phase_area_growth_s'] + plot_data2['phase_daughter_area_growth_s']
plot_data2['combined_motherbud_vol_growth'] = plot_data2['phase_volume_growth_s'] + plot_data2['phase_daughter_volume_growth_s']

In [ ]:
sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(x="phase_volume_growth_g1", y="combined_motherbud_vol_growth", data=plot_data2,
    hue="selection_subset_g1", height=10)
g._legend.set_title('Position Pool')
ax = plt.gca()
ax.set_ylabel("Combined Mother+Bud S growth [fL]", fontsize=20)
ax.set_xlabel("G1 growth [fL]", fontsize=20)
ax.set_title("G1 volume growth vs. mother+daughter cell growth in S phase", fontsize=30)
plt.show()

## Volume at birth vs. G1 duration (1st generation)
- Plot to determine if there is a negative correlation between cell size at birth and length of the first G1 phase

In [ ]:
# obtain table where one cell cycle is represented by one row: 
# first set of columns (like phase_length, growth...) for G1, second set of cols for S
plot_data3 = phase_grouped[phase_grouped.cell_cycle_stage=="G1"]
plot_data3 = plot_data3[plot_data3.complete_phase==1]
plot_data3 = plot_data3[plot_data3.generation_num==1]

sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(x="phase_volume_at_beginning", y="phase_length", data=plot_data3,
    hue="selection_subset", height=10)
g._legend.set_title('Position Pool')
ax = plt.gca()
ax.set_ylabel("Duration of first G1 phase [no of frames]", fontsize=20)
ax.set_xlabel("Volume at birth (first cytokinesis) [fL]", fontsize=20)
ax.set_title("Volume at birth vs G1 duration (1st generation)", fontsize=30)
plt.show()

## Volume at birth vs. Signal concentration at birth (1st generation)

In [ ]:
# set channel name here:
ch_name = 'mCitrine'
# obtain table where one cell cycle is represented by one row: 
# first set of columns (like phase_length, growth...) for G1, second set of cols for S
plot_data4 = phase_grouped[phase_grouped.cell_cycle_stage=="G1"]
plot_data4 = plot_data4[plot_data4.complete_phase==1]
plot_data4 = plot_data4[plot_data4.generation_num==1]

sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(x="phase_volume_at_beginning", y=f"phase_{ch_name}_concentration_at_beginning", data=plot_data4,
    hue="selection_subset", height=10, )
g._legend.set_title('Position Pool')
g.set(yscale="log")
ax = plt.gca()
ax.set_ylabel("mCitrine signal amount per volume in cell [a.u.]", fontsize=20)
ax.set_xlabel("Volume at birth (first cytokinesis) [fL]", fontsize=20)
ax.set_title("Volume at birth vs mCitrine signal amount per volume (1st generation)", fontsize=30)
plt.show()

## G1 vs. S duration (1st generation)

In [ ]:
# obtain table where one cell cycle is represented by one row: 
# first set of columns (like phase_length, growth...) for G1, second set of cols for S
complete_cc_data = phase_grouped[phase_grouped.all_complete==1]
s_data = complete_cc_data[complete_cc_data.cell_cycle_stage=="S"]
g1_data = complete_cc_data[complete_cc_data.cell_cycle_stage=="G1"]
plot_data1 = g1_data.merge(s_data, on=['Cell_ID', 'generation_num', 'position', 'file'], how='inner')
plot_data1 = plot_data1[plot_data1.generation_num==1]

sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(x="phase_length_x", y="phase_length_y", data=plot_data1,
    hue="selection_subset_x", height=10)
g._legend.set_title('Position Pool')
ax = plt.gca()
ax.set_ylabel("S duration same cycle [frames]", fontsize=20)
ax.set_xlabel("G1 duration [frames]", fontsize=20)
ax.set_title("G1 duration vs. S duration within same generation", fontsize=30)
plt.show()

# Plot gallery - zstack data, no timelapse

## Distribution of Cell volumes

In [ ]:
sns.set_theme(style="ticks", font_scale=2)

# Initialize the figure
plt.figure(figsize=(10,10))
sns.histplot(
    x='cell_vol_fl', 
    data=overall_df,
    hue='relationship',
    bins=20,
    legend=False
)
ax = plt.gca()
labels = [
    'Mother cells',
    'Buds'
]
handles = [
    mpatches.Patch(color=sns.color_palette('pastel')[0]),
    mpatches.Patch(color=sns.color_palette('pastel')[1])
]
ax.legend(
    handles=handles,
    labels=labels, 
    loc='upper right',
    #bbox_to_anchor = (1,0.2),
    framealpha=0.5
)

# Tweak the visual presentation
ax = plt.gca()
ax.set_xlabel("Cell volume [fL]", fontsize=20)
ax.set_title(f"Volume distribution, n: {overall_df.shape[0]}", fontsize=30)
#sns.despine(trim=True, left=True)
plt.show()

## Signal amounts calculated by mean projection vs. max projection

In [ ]:
# set channel name here:
ch_name = 'act1'
sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(
    x="act1_amount_autoBkgr_meanProj", 
    y="act1_amount_autoBkgr_maxProj", 
    data=overall_df,
    hue="relationship", 
    # hue='selection_subset', # try this if you selected multiple position pools
    height=10)
g._legend.set_title('Cell type')
ax = plt.gca()
ax.set_ylabel(f"{ch_name} signal amount (max projection of z-slices) [a.u.]", fontsize=20)
ax.set_xlabel(f"{ch_name} signal amount (mean projection of z-slices) [a.u.]", fontsize=20)
ax.set_title("Comparing mean projection with max projection", fontsize=30)
plt.show()

## Volume vs. Signal amount

In [ ]:
# set channel name here:
ch_name = 'act1'
sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(
    x="cell_vol_fl", 
    y=f"{ch_name}_amount_autoBkgr_meanProj", 
    data=overall_df,
    hue="relationship",
    # hue='selection_subset', # try this if you selected multiple position pools
    height=10
)
g._legend.set_title('Position Pool')
ax = plt.gca()
ax.set_ylabel(f"{ch_name} signal amount in cell [a.u.]", fontsize=20)
ax.set_xlabel("Cell volume [fL]", fontsize=20)
ax.set_title("Volume at birth vs signal amount", fontsize=30)
plt.show()

## Volume vs. Mean signal strength

In [ ]:
# set channel name here:
ch_name = 'act1'
sns.set_theme(style="darkgrid", font_scale=2)
# Initialize the figure
g = sns.lmplot(
    x="cell_vol_fl", 
    y=f"{ch_name}_mean_meanProj", 
    data=overall_df,
    hue="relationship", 
    height=10, 
)
g._legend.set_title('Cell type')
ax = plt.gca()
ax.set_ylabel(f"{ch_name} mean signal strength in cell [a.u.]", fontsize=20)
ax.set_xlabel("Cell volume [fL]", fontsize=20)
ax.set_title("Cell volume vs. mean signal strength", fontsize=30)
plt.show()

# ACDC paper figures

## Flurescence Signal over time (centered on bud emergence)
- timelapse data is assumed here
- note that in this plot, we filter for selection_subset==0
- make sure to change this if unwanted or to select the data with flu signal to be in the first pool

In [ ]:
# some configurations
# frame interval of video
frame_interval_minutes = 3
# quantiles of complete cell cycles (wrt phase lengths) to exclude from analysis 
# (not used, keep this for potential later use)
down_q, upper_q = 0, 1
# minimum number of cell cycles contributing to the mean+CI curve:
min_no_of_ccs = 10
# determine if you want to split the plot by generation
split_by_gen = True
# wether to scale to 0/1 or not
scale_data = False
# name of channel the signal of which should be plotted
ch_name = 'mCitrine'

In [ ]:
# select needed cols from overall_df_with_rel to not end up with too many columns
needed_cols = [
    'selection_subset', 'position', 'Cell_ID', 'cell_cycle_stage', 'generation_num', 'frame_i',
    f'{ch_name}_corrected_amount', f'{ch_name}_corrected_amount_rel', 
    'file', 'relationship', 'relative_ID', 'phase_length', 'phase_begin', f'gui_{ch_name}_amount_autoBkgr'
]
filter_idx = np.logical_and(overall_df_with_rel['complete_cycle'] == 1, overall_df_with_rel.selection_subset==0)
plot_data4a = overall_df_with_rel.loc[filter_idx, needed_cols].copy()
# calculate the time the cell already spent in the current frame at the current timepoint
plot_data4a['frames_in_phase'] = plot_data4a['frame_i'] - plot_data4a['phase_begin'] + 1
# calculate the time to the next (for G1 cells) and from the last (for S cells) G1/S transition  
plot_data4a['centered_frames_in_phase'] = plot_data4a.apply(
    lambda x: x.loc['frames_in_phase'] if\
    x.loc['cell_cycle_stage']=='S' else\
    x.loc['frames_in_phase']-1-x.loc['phase_length'],
    axis=1
)
# calculate combined signal and the "Pool, Phase ID" for the legend
# plot_data4a at this point only contains relationship==mother, 
# as generation_num==0 and relationship==bud are filtered out (incomplete cycle, cycles start with G1)
plot_data4a['Combined signal m&b'] = plot_data4a.apply(
    lambda x: x.loc[f'{ch_name}_corrected_amount']+x.loc[f'{ch_name}_corrected_amount_rel'] if\
    x.loc['cell_cycle_stage']=='S' and x.loc['relationship'] == 'mother' else\
    x.loc[f'{ch_name}_corrected_amount'],
    axis=1
)
plot_data4a['Bud signal'] = plot_data4a.apply(
    lambda x: x.loc[f'{ch_name}_corrected_amount_rel'] if\
    x.loc['cell_cycle_stage']=='S' and x.loc['relationship'] == 'mother' else 0,
    axis=1
)
# scale data if needed
if scale_data:
    maximum = max(
        plot_data4a['Combined signal m&b'].max(), 
        plot_data4a['Bud signal'].max()
    )
    plot_data4a['Combined signal m&b'] /= maximum
    plot_data4a['Bud signal'] /= maximum
# calculate min and max centered times per generation to eliminate up to a percentile
# (not used, as upper_q and lower_q are set to 100/0 respectively)
plot_data4a['min_centered_frames'] = plot_data4a.groupby(
    ['position', 'file', 'Cell_ID', 'generation_num']
)['centered_frames_in_phase'].transform(
    'min'
)
plot_data4a['max_centered_frames'] = plot_data4a.groupby(
    ['position', 'file', 'Cell_ID', 'generation_num']
)['centered_frames_in_phase'].transform(
    'max'
)
min_and_max = plot_data4a.groupby(
    ['Cell_ID', 'generation_num', 'position', 'file']
).agg(
    min_centered = ('min_centered_frames', 'first'),
    max_centered = ('max_centered_frames', 'first')
).reset_index()
min_val, max_val = np.quantile(
    min_and_max.min_centered, down_q
) * frame_interval_minutes, np.quantile(
    min_and_max.max_centered, upper_q
) * frame_interval_minutes
# perform selection (won't change anything if upper and lower are 100 and 0 respectively)
selection_indices = np.logical_and(
    plot_data4a.min_centered_frames*frame_interval_minutes>=min_val, 
    plot_data4a.max_centered_frames*frame_interval_minutes<=max_val
)
plot_data4a = plot_data4a[selection_indices]

# calculate centered time in minutes
plot_data4a['centered_time_in_minutes'] = plot_data4a.centered_frames_in_phase * frame_interval_minutes

# group dataframe to calculate sample sizes per generation
standard_grouped = plot_data4a.groupby(
    ['position', 'file', 'Cell_ID', 'generation_num']
).agg('count').reset_index()
plot_data4a['Generation'] = plot_data4a.apply(
    lambda x: f'1st ($n_1$={len(standard_grouped[standard_grouped.generation_num==1])})' if\
    x.loc['generation_num']==1 else f'2+ ($n_2$={len(standard_grouped[standard_grouped.generation_num>1])})',
    axis=1
)
if split_by_gen:
    g_cols = ['centered_frames_in_phase', 'Generation']
else:
    g_cols = 'centered_frames_in_phase'
plot_data4a['contributing_ccs_at_time'] = plot_data4a.groupby(g_cols).transform('count')['selection_subset']
plot_data4a = plot_data4a[plot_data4a.contributing_ccs_at_time >= min_no_of_ccs]

# finally prepare data for plot (use melt for multiple lines)
sample_size_4a = len(standard_grouped)
avg_cell_cycle_length = round(standard_grouped.loc[:,'centered_time_in_minutes'].mean())*frame_interval_minutes
cols_to_plot = ['Bud signal', 'Combined signal m&b']
index_cols = [col for col in plot_data4a.columns if col not in cols_to_plot]
plot_data4a_melted = pd.melt(
    plot_data4a, index_cols, var_name='Method of calculation'
).sort_values('Method of calculation')
data_dir = os.path.join('..', 'data', 'paper_plot_data')
# save preprocessed data for Fig. 4A
#plot_data4a_melted.to_csv(os.path.join(data_dir, 'plot_data4a_melted.csv'), index=False)
#plot_data4a.to_csv(os.path.join(data_dir, 'plot_data4a.csv'), index=False)

In [ ]:
# plot the data, comment out "style" argument to not make difference between generations
sns.set_theme(style="darkgrid", font_scale=1.6)
f, ax = plt.subplots(figsize=(15, 12))
if split_by_gen:
    style='Generation'
else:
    style=None
ax = sns.lineplot(
    data=plot_data6_melted,#.sort_values('Pool, Phase'),
    x="centered_time_in_minutes", 
    y="value",
    hue='Method of Calculation',
    #hue='position',
    style=style,
    ci=95
)
ax.axvline(x=0, color='red')#, label='Time of Bud Emergence')
ax.text(
    0.5, 0.21, "Time of \nBud Emergence", horizontalalignment='left', 
    size='medium', color='red', weight='normal'
)
ax.legend(
    title=f'Avg CC Length: {avg_cell_cycle_length} min, n = {sample_size}', 
    fancybox=True,
    labelspacing=0.5,
    handlelength=1.5,
    loc = 'upper left'
)
ax.set_ylabel("Total amount of Signal corrected by background [a.u.]", fontsize=20)
ax.set_xlabel("Time in phase relative to G1/S transition [minutes]", fontsize=20)
ax.set_title("Corrected Signal Amount during Cell Cycle Progression", fontsize=30)
plt.tight_layout()
plt.show()

## Volume at birth and division vs. mCitrine amount at birth (single cell) and division (combined)
- This plot is based on the grouped-by-phase data
- We assume that cell cycle phases were annotated with ACDC (~ column "cell_cycle_stage" exists, relative's dataframe was calculated and attached)

In [ ]:
# configure channel the signal of which should be plotted
ch_name = 'mCitrine'
# first set of columns (like phase_length, growth...) for G1, second set of cols for S
needed_cols = [
    'Cell_ID', 'generation_num', 'position', 'file', 'cell_cycle_stage', 'selection_subset', 
    'phase_volume_at_beginning', 'phase_volume_at_end', f'phase_{ch_name}_amount_at_beginning',
    f'phase_{ch_name}_combined_amount_at_end','phase_combined_volume_at_end'
]
plot_data4 = phase_grouped.loc[phase_grouped.complete_cycle==1, needed_cols]
scale_data = False

In [ ]:
plot_data4['relevant_volume'] = plot_data4.apply(
    lambda x: x.loc['phase_volume_at_beginning'] if\
    x.loc['cell_cycle_stage']=='G1' else\
    x.loc['phase_combined_volume_at_end'],
    axis=1
)
plot_data4['relevant_amount'] = plot_data4.apply(
    lambda x: x.loc[f'phase_{ch_name}_amount_at_beginning'] if\
    x.loc['cell_cycle_stage']=='G1' else\
    x.loc[f'phase_{ch_name}_combined_amount_at_end'],
    axis=1
)
# defining a function to generate entries for the figure legend 
# (assuming that selection_subset>0 is the autofluorescence control of the experiment)
def calc_legend_entry(x):
    if x.loc['selection_subset'] == 0:
        if x.loc['cell_cycle_stage']=='G1':
            return 'Single cell at birth'
        else:
            return 'Combined mother&bud at cytokinesis'
    else:
        if x.loc['cell_cycle_stage']=='G1':
            return 'Af control, single cell at birth'
        else:
            return 'Af control, combined mother&bud at cytokinesis'
        
plot_data4['Kind of Measurement'] = plot_data4.apply(
    lambda x: 'Single Cell in G1 (Frame after Cytokinesis)' if\
    x.loc['cell_cycle_stage']=='G1' else\
    'Combined Mother & Bud in S (Frame before Cytokinesis)',
    axis=1
)
plot_data4['Kind of Measurement new'] = plot_data4.apply(
    calc_legend_entry,
    axis=1
)
plot_data4['Generation'] = plot_data4.apply(
    lambda x: f'1st ($n_1$={int(len(plot_data4[plot_data4.generation_num==1])/2)})' if\
    x.loc['generation_num']==1 else f'2+ ($n_2$={int(len(plot_data4[plot_data4.generation_num>1])/2)})',
    axis=1
)
if scale_data:
    maximum = plot_data4['relevant_amount'].max()
    plot_data4['relevant_amount'] /= maximum
sample_size = len(plot_data4)


In [ ]:
#plot_data4 = plot_data4[plot_data4.selection_subset==1]
sns.set_theme(style="darkgrid", font_scale=1.6)
# create lmplot. Don't scatter and ommit legend to customize scatterplot and legend
sns.lmplot(
    x="relevant_volume", 
    y="relevant_amount", 
    data=plot_data4.sort_values(
        'Kind of Measurement new', ascending=False
    ),
    hue="Kind of Measurement new",
    legend=False,
    height=10,
    aspect=1.1,
    scatter=False
)
sns.scatterplot(
    x="relevant_volume", 
    y="relevant_amount", 
    data=plot_data4[plot_data4.generation_num==1].sort_values(
        'Kind of Measurement new', ascending=False
    ),
    hue="Kind of Measurement new",
    legend=False,
    marker='x'
)
sns.scatterplot(
    x="relevant_volume", 
    y="relevant_amount", 
    data=plot_data4[plot_data4.generation_num>1].sort_values(
        'Kind of Measurement new', ascending=False
    ),
    hue="Kind of Measurement new",
    legend=False,
    marker='o'
)
ax = plt.gca()
labels = [
    'Single cell at birth',
    'Combined mother&bud at cytokinesis',
    'Af control, single cell at birth',
    'Af control, combined mother&bud at cytokinesis',
    'Generation 1',
    'Generation 2+'
]
handles = [
    mpatches.Patch(color=sns.color_palette()[0]),
    mpatches.Patch(color=sns.color_palette()[1]),
    mpatches.Patch(color=sns.color_palette()[2]),
    mpatches.Patch(color=sns.color_palette()[3]),
    mlines.Line2D([], [], color='gray', marker='x', linestyle='None',
                          markersize=10),
    mlines.Line2D([], [], color='gray', marker='o', linestyle='None',
                          markersize=10)
]
ax.legend(
    handles=handles,
    labels=labels, 
    loc='center right',
    bbox_to_anchor = (1,0.2),
    framealpha=0.5
)
ax.set_ylabel("Amount of Signal in Cell(s) [a.u.]", fontsize=20)
ax.set_xlabel("Volume at Birth / Combined Volume Before Cytokinesis [fL]", fontsize=20)
ax.set_title(f"Volume at birth vs Signal Amount (n={int(sample_size/2)})", fontsize=30)
# format y-axis
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0), useMathText=True)
ax.get_yaxis().get_offset_text().set_position((-0.05,0))
# format x-axis
ax.set_xlim(0, plot_data4.relevant_volume.max()+20)
plt.tight_layout()
plt.show()
print(f'sample size flu-control: {len(plot_data4[plot_data4.selection_subset==1])//2}')
print(f'sample size tagged strain: {len(plot_data4[plot_data4.selection_subset==0])//2}')